In [1]:
import numpy as np
import numpy.matlib

import matplotlib.pyplot as plt
import matplotlib.pylab as plt
from matplotlib.pyplot import figure
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

import scipy.integrate
from scipy.integrate import solve_ivp
from scipy.sparse import csr_matrix, triu, spdiags
from scipy.sparse.linalg import spsolve, splu
import scipy.sparse
import scipy.optimize
from numpy.linalg import inv

import time

### Number 1

In [2]:
#initialize 
Time = 2
L = 10
n = 128 
x2 = np.linspace(-L, L, n+1)
x = x2[0:n]
dx = x[1]-x[0]

#time
t = np.linspace(0, Time, 501)
dt = t[1] - t[0]
time_steps = int(Time/dt)

CFL = (2*dt)/(dx**2) 
CFL

0.32768

### Number 2 (A1)

In [3]:
#von-Neumann analysis 

G = lambda l,z: (1/6)*(6-(15*l)+(16*l*np.cos(z))-(l*np.cos(2*z)))

# zvals = np.linspace(-np.pi,np.pi,128)
# lvals = np.linspace(0,3,128)
# [Z, L] = np.meshgrid(zvals,lvals)

# fig,ax = plt.subplots(figsize=(10,10),subplot_kw = {"projection":"3d"})
# ax.plot_surface(Z,L,G(L,Z), cmap = "magma")

In [4]:
A1 = np.abs(G(CFL,1))

In [5]:
A1

0.6756505247982867

### Number 3 (A2)

In [6]:
G_one = lambda z: np.abs(G(CFL,z))
x_optimize = scipy.optimize.fminbound(lambda z:-G_one(z), -np.pi,np.pi) # This gives the *maximizer*
A2 = G_one(x_optimize)

In [7]:
A2

1.0

### Number 4 (A3)

In [8]:
# n = 128
e1 = np.ones(n)
A = scipy.sparse.spdiags([-e1, -e1,16*e1,30*e1,16*e1,-e1, -e1], [-n+2,-2,-1,0,1,2,n-2],n,n, format = 'csc')
A[0,n-1] = 16
A[n-1,0] = 16

# p = 256
p = 256
e1 = np.ones(p)
A_256 = scipy.sparse.spdiags([-e1, -e1,16*e1,30*e1,16*e1,-e1, -e1], [-p+2,-2,-1,0,1,2,p-2],p,p, format = 'csc')
A_256[0,p-1] = 16
A_256[p-1,0] = 16

/Users/annahuang/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [9]:
A = A/12
A_256 = A_256/12

In [10]:
A.todense()

matrix([[ 2.5       ,  1.33333333, -0.08333333, ...,  0.        ,
         -0.08333333,  1.33333333],
        [ 1.33333333,  2.5       ,  1.33333333, ...,  0.        ,
          0.        , -0.08333333],
        [-0.08333333,  1.33333333,  2.5       , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  2.5       ,
          1.33333333, -0.08333333],
        [-0.08333333,  0.        ,  0.        , ...,  1.33333333,
          2.5       ,  1.33333333],
        [ 1.33333333, -0.08333333,  0.        , ..., -0.08333333,
          1.33333333,  2.5       ]])

In [11]:
A3 = A

In [12]:
A.shape

(128, 128)

In [13]:
A

<128x128 sparse matrix of type '<class 'numpy.float64'>'
	with 640 stored elements in Compressed Sparse Column format>

### Number 6 (A5)

In [15]:
#initialize 
Time = 2
L = 10
# n = 128 
x2 = np.linspace(-L, L, n+1)
x = x2[0:n]
dx = x[1]-x[0]

#time 
t = np.linspace(0, Time, 501)
dt = t[1] - t[0]
time_steps = int(Time/dt)

In [16]:
#Forward Euler 
def FE(n):
    usol_fe = np.zeros((len(x), len(t)))
    u0 = (10*np.cos((2*np.pi*x)/L) + 30*np.cos((8*np.pi*x)/L)).T
    usol_fe[:,0] = u0

    u1 = u0
    for j in range(time_steps):
        u2 = u1 + 0.5*CFL*A@u1
        u1 = u2
        usol_fe[:,j+1] = u2
    
    return usol_fe

In [17]:
# # plotting 
# fig, ax = plt.subplots(1,1)
# T, X = np.meshgrid(t,x)
# cp = ax.pcolor(x, t, FE(128).T, cmap='magma', shading='auto')
# fig.colorbar(cp)
# plt.show()

# # A lot of instability in this solution 

In [18]:
A5 = FE(128)[:,-1].reshape(128,1)

In [19]:
A5.shape

(128, 1)

### Number 7 (A6)

In [18]:
#von-Neumann analysis 

G1 = lambda l,z: (1-l+(l*np.cos(z)))/(1+l-(l*np.cos(z)))

# zvals = np.linspace(-np.pi,np.pi,128)
# lvals = np.linspace(0,3,128)
# [Z, L] = np.meshgrid(zvals,lvals)

# fig,ax = plt.subplots(figsize=(10,10),subplot_kw = {"projection":"3d"})
# ax.plot_surface(Z,L,G1(L,Z), cmap = "magma")

In [19]:
G_one1 = lambda z: np.abs(G1(CFL,z))
x_optimize1 = scipy.optimize.fminbound(lambda z:-G_one1(z), -np.pi,np.pi) # This gives the *maximizer*
A6 = G_one1(x_optimize1)

In [20]:
A6

1.0

### Number 8 (A7 & A8)

In [21]:
e1 = np.ones(n)
e1_p = np.ones(p)

M = scipy.sparse.spdiags([e1,-2*e1,e1], [-1,0,1],n,n, format = 'csc')
M_256 = scipy.sparse.spdiags([e1_p,-2*e1_p,e1_p], [-1,0,1],p,p, format = 'csc')

M[0,n-1] = 1
M[n-1,0] = 1

M_256[0,p-1] = 1
M_256[p-1,0] = 1

In [22]:
B = (scipy.sparse.eye(n) - ((CFL/2)*M))
C = (scipy.sparse.eye(n) + ((CFL/2)*M))

B_256 = (scipy.sparse.eye(p) - ((CFL/2)*M_256))
C_256 = (scipy.sparse.eye(p) + ((CFL/2)*M_256))

In [23]:
A7 = B.todense()
A8 = C.todense()

A7_256 = B_256.todense()
A8_256 = C_256.todense()

### Number 9 (A9)

In [24]:
#initialize 
Time = 2
L = 10
n = 128 
x2 = np.linspace(-L, L, n+1)
x = x2[0:n]
dx = x[1]-x[0]

#time 
t = np.linspace(0, Time, 501)
dt = t[1] - t[0]
time_steps = int(Time/dt)

In [25]:
def PLU(n):
    A = scipy.sparse.linalg.inv(B)*C
    PLU = scipy.sparse.linalg.splu(A)
    
    usol_plu = np.zeros((len(x),len(t))) #placeholder for solution - more effcient
    u0 = (10*np.cos((2*np.pi*x)/L) + 30*np.cos((8*np.pi*x)/L)).T
    usol_plu[:,0] = u0

    start = time.time()
    for j in range(time_steps):
        u1 = PLU.solve(u0)
        u0 = u1 
        usol_plu[:,j+1] = u1

    end = time.time()
    # print(end-start)
    
    return usol_plu

In [26]:
# #plotting x and t 
# fig,ax = plt.subplots(1,1)
# T,X = np.meshgrid(t,x)
# cp=ax.pcolor(x,t,PLU(128).T,cmap="magma",shading = 'auto')
# fig.colorbar(cp)
# plt.show()

In [27]:
A9 = PLU(128)[:,-1].reshape(128,1)

/Users/annahuang/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/Users/annahuang/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


### Number 10 (A10)

In [28]:
#initialize 
Time = 2
L = 10
n = 128 
x2 = np.linspace(-L, L, n+1)
x = x2[0:n]
dx = x[1]-x[0]

#time 
t = np.linspace(0, Time, 501)
dt = t[1] - t[0]
time_steps = int(Time/dt)

In [29]:
usol_bic =np.zeros((len(x),len(t))) #placeholder for solution - more effcient
u0 = (10*np.cos((2*np.pi*x)/L) + 30*np.cos((8*np.pi*x)/L)).T
usol_bic[:,0] = u0

start = time.time()

for j in range(time_steps):
    u1,info= scipy.sparse.linalg.bicgstab(A,u0,x0 = u0)
    u0=u1
    usol_bic[:,j+1] = u1

end = time.time()
# print(end-start)

# #plotting x and t 
# fig,ax = plt.subplots(1,1)
# T,X = np.meshgrid(t,x)
# cp=ax.pcolor(x,t,usol_bic.T,cmap="magma",shading = 'auto') # point out instability for Time = 10
# fig.colorbar(cp)
# plt.show()

In [30]:
A10 = usol_bic[:,-1].reshape(128,1)

### Number 11 (A11 & A12)

In [31]:
exact_128 = np.genfromtxt("exact_128.csv", delimiter=',')

In [32]:
A11 = np.linalg.norm(exact_128-A5)

In [33]:
A11

5.1091497362223745e+132

In [34]:
A12 = np.linalg.norm(exact_128-A9)

In [35]:
A12

nan

### Number 12 (A13 & A14)

In [36]:
#initialize 
Time = 2
L = 10
p = 256
x2 = np.linspace(-L, L, p+1)
x = x2[0:p]
dx = x[1]-x[0]

#time 
t = np.linspace(0, Time, 501)
dt = t[1] - t[0]
time_steps = int(Time/dt)

In [37]:
#Forward Euler 
usol_fe = np.zeros((len(x), len(t)))
u0 = (10*np.cos((2*np.pi*x)/L) + 30*np.cos((8*np.pi*x)/L)).T
usol_fe[:,0] = u0
    
u1 = u0
for j in range(time_steps):
    u2 = u1 + 0.5*CFL*A_256@u1
    u1 = u2
    usol_fe[:,j+1] = u2
     
# # plotting 
# fig, ax = plt.subplots(1,1)
# T, X = np.meshgrid(t,x)
# cp = ax.pcolor(x, t, usol_fe.T, cmap='magma', shading='auto')
# fig.colorbar(cp)
# plt.show()

A5_256 = usol_fe[:,-1].reshape(256,1)

In [38]:
exact_256 = np.genfromtxt("exact_256.csv", delimiter=',')

In [39]:
A13 = np.linalg.norm(exact_256-A5_256)

In [40]:
#initialize 
Time = 2
L = 10
p = 256
x2 = np.linspace(-L, L, p+1)
x = x2[0:p]
dx = x[1]-x[0]

#time 
t = np.linspace(0, Time, 501)
dt = t[1] - t[0]
time_steps = int(Time/dt)

In [45]:
A_256 = scipy.sparse.linalg.inv(B_256)*C_256
PLU_256 = scipy.sparse.linalg.splu(A_256)
    
usol_plu = np.zeros((len(x),len(t))) #placeholder for solution - more effcient
u0 = (10*np.cos((2*np.pi*x)/L) + 30*np.cos((8*np.pi*x)/L)).T
usol_plu[:,0] = u0

start = time.time()
for j in range(time_steps):
    u1 = PLU_256.solve(u0)
    u0 = u1 
    usol_plu[:,j+1] = u1

end = time.time()
# print(end-start)

A9_256 = usol_plu[:,-1].reshape(256,1)

In [46]:
A14 = np.linalg.norm(exact_256-A9_256)